# Building a Catalog

In [ ]:
from classes.source_catalog import GetCnilCatalog

url = 'https://www.data.gouv.fr/api/1/organizations/534fff61a3a7292c64a77d59/catalog'
headers = {'accept': 'application/json'}
url_add = 'https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv'
instance1 = GetCnilCatalog(url, headers, url_add)
data = instance1.fetch_data_from_api()
data = data['@graph']
table_name = 'title'
download_url = 'downloadURL'
table_id = 'identifier' 
file_format= 'format'
last_update= 'modified' 
accessURL = '@id'
df_catalog = instance1.response_to_dataframe(data=data, table_name=table_name, download_url=download_url, table_id=table_id, file_format=file_format, last_update=last_update, accessURL=accessURL)
df_dataset = instance1.load_additional_info()
df_catalog = instance1.identify_datasets_info()
df_catalog = instance1.merge_additional_info()
instance1.save_to_csv('source_cnil_catalog')

# Uploading Files to GCS

In [ ]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv','data/catalog/opencnil-violationsdcpnotifiees-20230930.csv']
dest_folder = 'raw'
dest_blob = ['source_cnil_catalog_2024-02-10.csv', 'opencnil-violationsdcpnotifiees-20230930.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

In [ ]:
bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
url = ['https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv', 'https://www.data.gouv.fr/fr/datasets/r/0f678674-4327-4c4d-8819-b6f508b41d0e']
dest_folder = 'raw'
dest_blob = ['datasets.csv', 'plaintes.csv']
init2.download_and_upload_from_URLs(url, dest_folder, dest_blob)

# Downloading from Catalog

In [ ]:
from classes.download_catalog_content import DlCatalogContent

instance3 = DlCatalogContent('data/catalog/source_cnil_catalog_2024-02-10.csv')
instance3.get_tables()
instance3.zip_files()

# Prep data to upload to BQ

In [ ]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv','data/raw_datasets.zip']
dest_folder = 'raw'
init2.local_to_gcs(file_paths, dest_folder)

In [ ]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-12/raw/raw_datasets.zip'
output_folder_name = '2024-02-12/'+ 'prep'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
instance4.process_zip_file()

In [ ]:
import pandas as pd

df = pd.read_excel('data/raw_datasets/controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2014_v2015-06-15')
df

In [1]:
from classes.list_files import FolderLister

instance6 = FolderLister('data/raw_datasets')
# instance6.list_folders()
paths = instance6.list_rel_paths()
print(paths)

['data/raw_datasets/budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv', 'data/raw_datasets/budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.xlsx', 'data/raw_datasets/traitements-de-donnees-personnelles-declares-a-la-cnil-depuis-le-25-mai-2018/Formalités préalables reçues par la CNIL depuis le 25 mai 2018_v2024-02-05', 'data/raw_datasets/marches-publics-de-la-cnil/opencnil-marches-publics-2014-2020_v2021-06-02.xlsx', 'data/raw_datasets/mises-en-demeure-prononcees-par-la-cnil/open-cnil-volumes-med-depuis-2014-maj-aout-2023_v2023-08-25.csv', 'data/raw_datasets/mises-en-demeure-prononcees-par-la-cnil/open-cnil-volumes-med-depuis-2014-maj-aout-2023_v2023-08-25.xlsx', 'data/raw_datasets/effectifs-de-la-cnil/opencnil-effectifs-depuis-1980-maj-juin-2023_v2023-06-28.csv', 'data/raw_datasets/effectifs-de-la-cnil/opencnil-effectifs-depuis-1980-maj-juin-2023_v2023-06-28.xlsx', 'data/raw_datasets/les-deliberations-de-la-cnil/DTD CNIL _v2023-11-17

In [2]:
from classes.prep_data import PrepFilesBQ

instance5 = PrepFilesBQ(paths)
instance5.process_files()

data/raw_datasets/budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv
extension found
read_csv with , shape : (6, 1) columns Index(['Année ;2000;2001;2002;2003;2004;2005;2006;2007;2008;2009;2010;2011;2012;2013;2014;2015;2016;2017;2018;2019;2020;2021;2022;2023'], dtype='object')
  Année ;2000;2001;2002;2003;2004;2005;2006;2007;2008;2009;2010;2011;2012;2013;2014;2015;2016;2017;2018;2019;2020;2021;2022;2023
0  Budget total;4 971 196;6 148 190;6 416 811;6 4...                                                                            
1  dont personnel (titre 2);3 344 612;3 858 057;4...                                                                            
2  dont fonctionnement;1 626 584;2 290 133;2 296 ...                                                                            
3                           ;;;;;;;;;;;;;;;;;;;;;;;;                                                                            
4                           ;;;;;;;;;;;;;;;;;;;;;;

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


read_excel shape : (496, 6) columns Index(['Année', 'Type de contrôle', 'Organismes', 'Lieu de contrôle',
       'Département', 'Secteur d'activité \nde l'organisme'],
      dtype='object')
   Année Type de contrôle                Organismes Lieu de contrôle  \
0   2015           loi 78    BOURSE DE L'IMMOBILIER         Bordeaux   
1   2015           loi 78              MGI-TWC SAS          Les Fins   
2   2015           loi 78  SPORTSDIRECT.COM FRANCE           Fresnes   
3   2015           loi 78                 AUTOLIB'        Vaucresson   
4   2015           loi 78            ASSURLAND.COM             Paris   

   Département Secteur d'activité \nde l'organisme  
0           33                          Immobilier  
1           25                            Commerce  
2           94                            Commerce  
3           92                           Transport  
4           75                            Commerce  
No need to transpose
The column_formatter method worked per

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


read_excel shape : (384, 7) columns Index(['Catégorie\nde contrôle', 'Modalité\nde contrôle',
       'Organismes contrôlés', 'Dépt.', 'Ville', 'Pays',
       'Activité\nde l'organisme'],
      dtype='object')
  Catégorie\nde contrôle Modalité\nde contrôle  Organismes contrôlés  Dépt.  \
0                   RGPD              en ligne           BRICO PRIVE   31.0   
1                   RGPD            sur pièces              CLUSTREE   75.0   
2                   RGPD          sur audition        MEDIAN CONSEIL   64.0   
3                   RGPD             sur place  MINISTÈRE DES ARMÉES   30.0   
4                   RGPD              en ligne             LE FIGARO   75.0   

     Ville      Pays Activité\nde l'organisme  
0  L'UNION  FRANCE                   Commerce  
1    PARIS  FRANCE                    Travail  
2      PAU  FRANCE                     Santé   
3    NIMES  FRANCE                   Régalien  
4    PARIS  FRANCE                   Commerce  
No need to transpose
The col

...

read_excel shape : (96524, 26) columns Index(['Extraction du 29/01/2024', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')
    Extraction du 29/01/2024  \
0  SIREN organisme désignant   
1                  110000122   
2                  180046013   
3                  200052264   
4                  718201742   

                                          Unnamed: 1  \
0                            Nom organisme désignant   
1  COMMISSION NATIONALE DE L'INFORMATIQUE ET DES ...   
2                     CENTRE DES MONUMENTS NATIONAUX   
3                                   REGION GRAND EST   
4             SOCIETE 

Skipping line 10004: expected 26 fields, saw 27
Skipping line 21170: expected 26 fields, saw 27
Skipping line 21826: expected 26 fields, saw 27
Skipping line 24967: expected 26 fields, saw 27

Skipping line 93338: expected 26 fields, saw 27



read_csv with ; shape : (96518, 26) columns Index(['SIREN organisme désignant', 'Nom organisme désignant',
       'Secteur activité organisme désignant', 'Code NAF organisme désignant',
       'Adresse postale organisme désignant',
       'Code postal organisme désignant', 'Ville organisme désignant',
       'Pays organisme désignant', 'Type de DPO', 'Date de la désignation',
       'SIREN organisme désigné', 'Nom organisme désigné',
       'Secteur activité organisme désigné', 'Code NAF organisme désigné',
       'Adresse postale organisme désigné', 'Code postal organisme désigné',
       'Ville organisme désigné', 'Pays organisme désigné',
       'Moyen contact DPO email', 'Moyen contact DPO url',
       'Moyen contact DPO téléphone', 'Moyen contact DPO adresse postale',
       'Moyen contact DPO code postal', 'Moyen contact DPO ville',
       'Moyen contact DPO pays', 'Moyen contact DPO autre'],
      dtype='object')
  SIREN organisme désignant  \
0                 110000122   
1   

Skipping line 6: expected 1 fields, saw 3
Skipping line 7: expected 1 fields, saw 16
Skipping line 130: expected 1 fields, saw 2
Skipping line 151: expected 1 fields, saw 2
Skipping line 197: expected 1 fields, saw 5
Skipping line 211: expected 1 fields, saw 5
Skipping line 214: expected 1 fields, saw 5
Skipping line 217: expected 1 fields, saw 5
Skipping line 220: expected 1 fields, saw 6
Skipping line 223: expected 1 fields, saw 5
Skipping line 445: expected 1 fields, saw 5
Skipping line 451: expected 1 fields, saw 3
Skipping line 455: expected 1 fields, saw 2
Skipping line 655: expected 1 fields, saw 5
Skipping line 676: expected 1 fields, saw 5
Skipping line 680: expected 1 fields, saw 5
Skipping line 684: expected 1 fields, saw 5
Skipping line 688: expected 1 fields, saw 6
Skipping line 692: expected 1 fields, saw 5
Skipping line 730: expected 1 fields, saw 2
Skipping line 739: expected 1 fields, saw 3
Skipping line 748: expected 1 fields, saw 3
Skipping line 791: expected 1 field

data/raw_datasets/notifications-a-la-cnil-de-violations-de-donnees-a-caractere-personnel/opencnil-violationsdcpnotifiees-20230930_v2023-12-21.xlsx
extension found
read_excel shape : (18805, 9) columns Index(['Extraction générée le 20 décembre 2023', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8'],
      dtype='object')
  Extraction générée le 20 décembre 2023  \
0  Date de réception de la notification    
1                                2023-09   
2                                2023-09   
3                                2023-09   
4                                2023-09   

                                          Unnamed: 1  \
0         Secteur d'activité de l'organisme concerné   
1                            Administration publique   
2  Activités spécialisées, scientifiques et techn...   
3                       Information et communication   
4                       Information et communication   


In [3]:
import pandas as pd

df = pd.read_csv('data/raw_datasets/effectifs-de-la-cnil/opencnil-effectifs-depuis-1980-maj-juin-2023_v2023-06-28.csv')
df.head()

,Année ;1980;1981;1982;1983;1984;1985;1986;1987;1988;1989;1990;1991;1992;1993;1994;1995;1996;1997;1998;1999;2000;2001;2002;2003;2004;2005;2006;2007;2008;2009;2010;2011;2012;2013;2014;2015;2016;2017;2018;2019;2020;2021;2022;2023
0,Postes budgétaires;28;28;32;32;34;34;37;37;37;...
1,ETPT;;;;;;;;;;;;;;;;;;;;;;;;;80;85;90;100;113;...
